## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import time
import numpy as np
import pandas as pd
import codecs
import lxml
import csv
import psutil
import os.path

## Constante variaible

In [2]:
urls_txt = "https.txt"
folder = 'Task 1.2/dataset'
folder2 = 'Task 1.3/anime_tsv'
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36",
    'accept': "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    'referer': "https://myanimelist.net/"
} 
save_path = 'simplePath'

## Q 1.1

In [4]:
c = 0
reg = r'href=\"https://[\w\W]*" id'
htt = list()
for i in range(400):
    cnt = requests.get("https://myanimelist.net/topanime.php?limit="+str(i*50))
    soup = BeautifulSoup(cnt.content, features="lxml")
    links = soup.find_all('h3')
    for l in range(len(links)-3):
        li = links[l].find('a')
        m = re.findall(reg, str(li))
        s = m[0][6:][:-4]+'\n'
        htt.append(s)
f = open("urls_txt", 'w')
for i in htt:
    f.write(i)
f.close

<function TextIOWrapper.close()>

## Q 1.2

In [96]:
def htmls_by_urls(urls_txt, folder):
     # urls_txt: string 'https.txt' from previous task
    # folder: string; eg '/Users/anton/Desktop/ADM/Homework3/html'
    
    with open(urls_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # list of urls
    list_txt = [line.strip() for line in lines]
    
    i = 0
    
    while i < len(list_txt):
        url = list_txt[i]
        # folder where we save html
        al_folder = '{}/page_{}/{}.html'.format(folder, i//50 +1, i+1)
        # download html
        html = requests.get(url, headers)
        if(html.status_code != 200) : 
            time.sleep(120)
            print('error', html.status_code)
        else:
            i += 1
            with open(al_folder, 'w', encoding='utf-8') as g:
                g.write(html.text)


In [97]:
htmls_by_urls(urls_txt, folder)

error 403
error 403
error 403
error 403


## Q 1.3

In [27]:
arr = os.listdir(folder)
alarr = list()
for y in arr:
    if "rar" in y:
        continue
    fiarr = os.listdir(folder+'/'+y)
    if '.ipynb_checkpoints' in fiarr:
        fiarr.remove('.ipynb_checkpoints')
    for i in range(len(fiarr)):
        fiarr[i] = y+'/'+fiarr[i]
    alarr.extend(fiarr)
print(len(alarr))

19053


In [ ]:
reg1 = r'\n[\w\W]*to'
reg2 = r'to[\w\W]*\n'
trfile = 'file.csv'
animeTitle = []
animeTypes = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = [] 
df = pd.DataFrame(
[animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
bs = 9
for fiName in range(len(alarr)):
    # anime titles
    path = folder+'/'+alarr[fiName]
    #print(path)
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left sied of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # anime types
            if span.contents[0] == 'Type:':
                tr = div.find_all('a')
                if len(tr) ==0:
                    animeTypes.append(div.contents[2][2:])
                else:
                    animeTypes.append(div.find_all('a')[0].contents[0])
            # anime number of episodes
            if span.contents[0] == 'Episodes:':
                try :
                    animeNumEpisode.append(int(div.contents[2]))
                except:
                    animeNumEpisode.append(0)
            # anime release and end dates
            if span.contents[0] == 'Aired:':
                if 'Not available' in div.contents[2]:
                    releaseDate.append('?')
                    endDate.append('?')
                elif 'to' in div.contents[2]:                    
                    if "?" in re.findall(reg2, str(div.contents[2]))[0][3:-1]:
                        endDate.append("?")
                    else:
                        endDate.append(pd.to_datetime(re.findall(reg2, str(div.contents[2]))[0][3:-1]))
                    if "?" in re.findall(reg1, str(div.contents[2]))[0][3:-1]:
                        releaseDate.append("?")
                    else:
                        releaseDate.append(pd.to_datetime(re.findall(reg1, str(div.contents[2]))[0][2:-3]))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][2:-2]))
                    endDate.append('-')
                    
    # middle side of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        # anime number of members
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        # anime score
        score = div.find_all("div", {"class": "score-label score-"+str(bs)})
        while len(score) == 0:
            bs -= 1
            score = div.find_all("div", {"class": "score-label score-"+str(bs)})
            if bs == -1:
                bs = 10
                score = div.find_all("div", {"class": "score-label score-na"})
        try:        
            animeScore.append(float(score[0].contents[0]))
            # anime number of users
            users = div.find_all("div", {"class": "fl-l score"})
            animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        except:
            animeScore.append(None)
            animeUsers.append(0)

        # anime rank
        rank = div.find_all("span", {"class": "numbers ranked"})
        try:
            animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        except:
            animeRank.append(-1)
        # anime popularity
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
        
    # anime description
    description = soup.find_all("p", {"itemprop": "description"})
    for br in description[0].find_all("br"):
        br.replace_with("\n")
    animeDescription.append(description[0].contents)
    
    # related animes 
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    x = []
    y = []
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            try:
                t = td[i+1].find_all("a")
                y.append(t[0].contents[0])
            except:
                y.append('NA')
            
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
        if(len(animeRelated)<len(animeDescription)):
            animeRelated.append("None")
    try:    
        # anime characters and voices
        characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
        x = []
        y = []
        voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
        for i in chars:
            x.append(i.contents[0].contents[0])
        for i in voices:
            y.append(i.contents[1].contents[0])
        animeCharacters.append(x)
        animeVoices.append(y)
    except:
        animeCharacters.append("NA")
        animeVoices.append("NA")
    
    # anime staff
    try:
        staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        staff = staff[1].find_all("td")
        x = []
        y = []
        for i in range(1, len(staff), 2):
            x.append(staff[i].contents[1].contents[0])
            y.append(staff[i].find_all("small")[0].contents[0])
        animeStaff.append([list(i) for i in list(zip(x,y))])
    except:
        animeStaff.append("NA")
        
        
    # Following lines are made to save memory by storing the current info in a pd instead of the array    
    if ((fiName+1)%500) == 0:
        print(path,fiName)
        dftr = pd.DataFrame(
        [animeTitle.copy(), animeTypes.copy(), animeNumEpisode.copy(), releaseDate.copy(), endDate.copy(), animeNumMembers.copy(), animeScore.copy(), animeUsers.copy(), animeRank.copy(), animePopularity.copy(), animeDescription.copy(), animeRelated.copy(), animeCharacters.copy(), animeVoices.copy(), animeStaff.copy()], 
        index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
        ).T
        df = pd.concat([df, dftr])
        df.to_csv(trfile, index=False)
        del df, dftr
        animeTitle = []
        animeTypes = []
        animeNumEpisode = []
        releaseDate = []
        endDate = []
        animeNumMembers = []
        animeScore = []
        animeUsers = []
        animeRank = []
        animePopularity = []
        animeDescription = []
        animeRelated = []
        animeCharacters = []
        animeVoices = []
        animeStaff = [] 
        df = pd.read_csv(trfile)
        os.remove(trfile)

Task 1.2/dataset/page_107/5350.html 499
Task 1.2/dataset/page_116/5800.html 999
Task 1.2/dataset/page_125/6250.html 1499
Task 1.2/dataset/page_134/6700.html 1999
Task 1.2/dataset/page_143/7150.html 2499
Task 1.2/dataset/page_152/7600.html 2999
Task 1.2/dataset/page_161/8050.html 3499
Task 1.2/dataset/page_170/8500.html 3999
Task 1.2/dataset/page_18/900.html 4499
Task 1.2/dataset/page_189/9450.html 4999
Task 1.2/dataset/page_198/9900.html 5499
Task 1.2/dataset/page_206/10300.html 5999
Task 1.2/dataset/page_215/10750.html 6499
Task 1.2/dataset/page_224/11200.html 6999
Task 1.2/dataset/page_233/11650.html 7499
Task 1.2/dataset/page_242/12100.html 7999
Task 1.2/dataset/page_251/12550.html 8499
Task 1.2/dataset/page_260/13000.html 8999
Task 1.2/dataset/page_27/1350.html 9499
Task 1.2/dataset/page_279/13950.html 9999
Task 1.2/dataset/page_288/14400.html 10499
Task 1.2/dataset/page_297/14850.html 10999
Task 1.2/dataset/page_305/15250.html 11499
Task 1.2/dataset/page_314/15700.html 11999
Task 

In [ ]:
dftr = pd.DataFrame(
[animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
df = pd.concat([df, dftr])
df.to_csv(trfile, index=False)
del df, dftr, animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff
df = pd.read_csv(trfile)

In [ ]:
df

In [ ]:
#For each row I created a tsv file with the value of each anime 
for i in range(len(df)):
    with open(folder2+'/anime_'+str(i)+'.tsv', 'wt') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in df.columns]) #the header row
        tsv_writer.writerow(x for x in df.iloc[i]) #the value under each columns
        

In [ ]:
df.to_csv(trfile, index=False)